In [1]:
from collections import Counter
from enum import IntEnum
from pathlib import Path
import shutil
import subprocess
from tqdm import tqdm
from typing import Dict, List, Optional

Use examples in [python cookbook, 3rd Edition](https://www.amazon.com/Python-Cookbook-Third-David-Beazley/dp/1449340377) as test cases to verify the correctness of the implementation.

In [2]:
IWASM = Path(shutil.which("iwasm"))

PY2WASM = Path(shutil.which("py2wasm"))

PYTHON_COOKBOOK = Path.cwd().joinpath("./cookbook/src")
assert PYTHON_COOKBOOK.exists(), "use `git submodule update` to fetch cookbook"


In [3]:
class CaseErrorCode(IntEnum):
    OK = 0
    COMPILATION_FAILURE = 1
    EXECUTION_FAILURE = 2
    DIFFERENT_RESULT = 3
    BYPASS = 4

    def __str__(self):
        return f'{self.name}'

class CaseResult:
    def __init__(self, error_code: CaseErrorCode, msg: str):
        self.error_code = error_code
        self.msg = msg

    def __repr__(self):
        return f"{self.error_code}\n{self.msg}"


For each case, we are going to:

1. compile the python script to a WebAssembly core module 
2. run the python script 
3. run the WebAssembly core module
4. compare both execution results of step 2 and step 3

In [20]:
def compile_py_2_wasm(py2wasm_bin: Path, py_file: Path, out_dir: Optional[Path]) -> Path:
    if not out_dir is None:
        out_dir.mkdir(parents=True, exist_ok=True)
        wasm_file = out_dir.joinpath(py_file.stem + ".wasm")
    else:
        wasm_file = py_file.with_suffix(".wasm")
    
    cmd = f"{py2wasm_bin} {py_file} -o {wasm_file}"
    subprocess.run(cmd, cwd=Path.cwd(), capture_output=True, check=True, shell=True, universal_newlines=True, text=True)
    return wasm_file

def execute_py(py_file: Path) -> subprocess.CompletedProcess:
    """
    Since relative pathes are used in examples.
    """
    cwd = py_file.parent
    cmd = f"python {py_file}"
    return subprocess.run(cmd, cwd=cwd, capture_output=True, check=True, shell=True, universal_newlines=True, text=True)

def execute_wasm(iwasm_bin: Path, wasm_file: Path) -> subprocess.CompletedProcess:
    """
    To avoid the WASI pre-opens problem, run under the same directory with the wasm file
    """
    cwd = wasm_file.parent
    cmd = f"{iwasm_bin} --dir=. {wasm_file.relative_to(cwd)}"
    return subprocess.run(cmd, cwd=cwd, capture_output=True, check=True, shell=True, universal_newlines=True, text=True)
    

def compare_result(expected: str, actual: str) -> bool:
    return expected == actual

##################################################
def execute_case(py_file: Path, out_dir: Optional[Path]) -> CaseResult:
    try:
        wasm_file = compile_py_2_wasm(PY2WASM, py_file, out_dir)
    except subprocess.CalledProcessError:
        return CaseResult(CaseErrorCode.COMPILATION_FAILURE, "")

    try:
        p = execute_wasm(IWASM, wasm_file)
    except subprocess.CalledProcessError as e:
        return CaseResult(CaseErrorCode.EXECUTION_FAILURE, e.stderr if e.stderr else e.stdout)

    output_from_wasm = p.stdout

    p = execute_py(py_file)
    output_from_py = p.stdout

    if not compare_result(output_from_py, output_from_wasm):
        return CaseResult(CaseErrorCode.DIFFERENT_RESULT, f"{'>' * 40}\n{output_from_py}{'-' * 20} V.S. {'-' * 20}\n{output_from_wasm}{'<' * 40}")
    
    return CaseResult(CaseErrorCode.OK, "")

def visit_case(py_file: Path, out_dir: Optional[Path]) -> CaseResult:
    print(f"py_file={py_file}, out_dir={out_dir}")
    return CaseResult(CaseErrorCode.OK, "")

def execute_test(chapter: Path, bypass: List[str]) -> Dict[str, CaseResult]:
    # list all sub-directories in chapter
    chapter_tests = [x for x in chapter.iterdir() if x.is_dir()]

    # walk
    ret = {}
    for test_dir in tqdm(chapter_tests):
        if test_dir.name in bypass:
            ret[case_name] = CaseResult(CaseErrorCode.BYPASS, "")
            continue

        # assume all .py under the test_dir are test cases
        py_files = [x for x in test_dir.iterdir() if x.suffix == ".py"]

        for py_file in py_files:
            result = execute_case(py_file, None)
            # result = visit_case(py_file, None)
            case_name = f"{test_dir.name}.{py_file.stem}"
            ret[case_name] = result
    
    return ret

def summarize_result(results: Dict[str, CaseResult]) -> None:
    counter = Counter([x.error_code for x in results.values()])
    print()
    print(counter)
    print()

    # print failures
    for k,v in results.items():
        if v.error_code != CaseErrorCode.OK:
            print(f"{k} FAILED:\n{v}")
            print()

## Chapter 1. Data Structures And Algorithms

`DIFFERENT_RESULT` in *finding_out_what_two_dictionaries_have_in_common* isn't a bug 😄. It is caused by the different order of the keys in the dictionary.

`EXECUTION_FAILURE` in *transforming_and_reducing_data_at_the_same_time* needs more investigation 🔎. My first guess is about preopen.

In [116]:
chapter_1 = PYTHON_COOKBOOK.joinpath("1")
chapter_1_result = execute_test(chapter_1, [])
summarize_result(chapter_1_result)

100%|██████████| 16/16 [04:38<00:00, 17.40s/it]

Counter({<CaseErrorCode.OK: 0>: 15, <CaseErrorCode.DIFFERENT_RESULT: 3>: 1, <CaseErrorCode.EXECUTION_FAILURE: 2>: 1})

finding_out_what_two_dictionaries_have_in_common.example FAILED:
DIFFERENT_RESULT
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Common keys: {'y', 'x'}
Keys in a not in b: {'z'}
(key,value) pairs in common: {('y', 2)}
-------------------- V.S. --------------------
Common keys: {'x', 'y'}
Keys in a not in b: {'z'}
(key,value) pairs in common: {('y', 2)}
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

transforming_and_reducing_data_at_the_same_time.example FAILED:
EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example.py", line 6, in <module>
    files = os.listdir(os.path.expanduser('~'))
    
FileNotFoundError: [Errno 44] No such file or directory: '~'




### Investigation of *transforming_and_reducing_data_at_the_same_time*

In [120]:
! iwasm --dir=. --dir=/home/vscode ./cookbook/src/1/transforming_and_reducing_data_at_the_same_time/example.wasm

Traceback (most recent call last):
  File "./example.py", line 6, in <module>
    files = os.listdir(os.path.expanduser('~'))
    
FileNotFoundError: [Errno 44] No such file or directory: '~'


In [121]:
! iwasm --dir=. --dir=~ ./cookbook/src/1/transforming_and_reducing_data_at_the_same_time/example.wasm

error while pre-opening directory ~: 2



## Chapter 2. Strings And Text

There are `try...except...` in some cases. Need to deep dive into the wasm implementation.

In [21]:
chapter_2 = PYTHON_COOKBOOK.joinpath("2")
chapter_2_result = execute_test(chapter_2, [])
summarize_result(chapter_2_result)

100%|██████████| 13/13 [04:36<00:00, 21.29s/it]


Counter({<CaseErrorCode.OK: 0>: 14, <CaseErrorCode.EXECUTION_FAILURE: 2>: 1})

writing_a_simple_recursive_descent_parser.plyexample FAILED:
EXECUTION_FAILURE
Exception: indirect call type mismatch


